# 💬 ChipChat - 데이터시트 챗봇

**전처리된 데이터시트 JSON 파일을 기반으로 질의응답을 수행하는 챗봇입니다.**

---

## 🚀 시작하기 전에

이 노트북은 ChipChat 앱을 실행하는 방법을 제공합니다.
로컬 환경과 Google Colab 환경 모두에서 사용할 수 있습니다.

## ✨ 주요 기능들
- 🤖 **AI Agent 시스템**: LangGraph 기반 스마트 에이전트
- 🔧 **3가지 Tool 자동 선택**: chipDB 검색, 벡터스토어 검색, PDF 처리
- 📊 **ChipDB.csv 연동**: 부품 사양 요약 데이터베이스 활용
- 🎯 **다중 LLM 지원**: OpenAI와 Claude 모델 선택 가능
- 📄 **실시간 PDF 업로드**: 새 데이터시트 자동 처리 및 통합
- 🔍 **고급 필터링**: 부품번호, 제조사, 카테고리별 검색
- 🛠️ **프롬프트 커스터마이징**: 시스템 프롬프트 자유 수정
- 🏷️ **메타데이터 추적**: 소스 정보 표시

## 📋 사용 방법
아래 셀들을 **순서대로 실행**만 하면 됩니다. (각 셀의 ▶️ 버튼을 클릭)

---

In [ ]:
#@title ✅ 1단계: 환경 설정 { display-mode: "form" }
#@markdown 로컬 또는 Google Drive 환경을 선택하고 설정을 관리합니다.

import os
import json
from pathlib import Path

# 환경 선택 토글
use_google_drive = False #@param {type:"boolean"}
#@markdown ✅ 체크: Google Drive 사용 (Colab 환경) | ❌ 체크해제: 로컬 환경 사용

# config.json 파일 경로
config_file = Path('config.json')

if config_file.exists():
    # 기존 config.json 로드
    with open(config_file, 'r', encoding='utf-8') as f:
        config = json.load(f)
    
    # 환경 설정 업데이트
    config['environment']['use_google_drive'] = use_google_drive
    
    # 업데이트된 설정 저장
    with open(config_file, 'w', encoding='utf-8') as f:
        json.dump(config, f, indent=4, ensure_ascii=False)
    
    print("✅ config.json 업데이트 완료!")
else:
    print("❌ config.json 파일이 없습니다. 기본 설정을 생성합니다.")
    
    # 기본 config.json 생성
    default_config = {
        "environment": {
            "use_google_drive": use_google_drive,
            "description": "Environment selection: true for Google Drive (Colab), false for local"
        },
        "paths": {
            "google_drive": {
                "base_path": "/content/drive/MyDrive",
                "prep_json_folder": "/content/drive/MyDrive/prep_json",
                "vectorstore_folder": "/content/drive/MyDrive/vectorstore",
                "prompt_templates_folder": "/content/drive/MyDrive/prompt_templates",
                "model_cache_folder": "/content/drive/MyDrive/hf_model_cache",
                "logs_folder": "/content/drive/MyDrive/chipchat_logs"
            },
            "local": {
                "base_path": ".",
                "prep_json_folder": "./prep_json",
                "vectorstore_folder": "./vectorstore",
                "prompt_templates_folder": "./prompt_templates",
                "model_cache_folder": "./hf_model_cache",
                "logs_folder": "./logs"
            }
        },
        "models": {
            "embedding_model": "sentence-transformers/all-MiniLM-L6-v2",
            "supported_llm": {
                "openai": ["gpt-4o-mini", "gpt-4o", "gpt-3.5-turbo"],
                "claude": ["claude-3-sonnet", "claude-3-haiku", "claude-3-opus"]
            }
        }
    }
    
    with open(config_file, 'w', encoding='utf-8') as f:
        json.dump(default_config, f, indent=4, ensure_ascii=False)
    
    config = default_config
    print("✅ 기본 config.json 생성 완료!")

# 현재 환경 설정 표시
env_type = "Google Drive" if use_google_drive else "로컬"
current_paths = config['paths']['google_drive' if use_google_drive else 'local']

print(f"\n🌐 {env_type} 환경으로 설정되었습니다!")
print(f"📋 설정된 경로들:")
for key, path in current_paths.items():
    if key != 'base_path':
        print(f"  • {key.replace('_', ' ').title()}: {path}")

# Google Drive 마운트 (필요한 경우)
if use_google_drive:
    try:
        from google.colab import drive
        print("\n🔄 Google Drive 마운트 중...")
        drive.mount('/content/drive')
        print("✅ Google Drive 연동 완료!")
    except ImportError:
        print("⚠️ Google Colab 환경이 아닙니다. 환경 설정을 로컬로 변경합니다.")
        config['environment']['use_google_drive'] = False
        with open(config_file, 'w', encoding='utf-8') as f:
            json.dump(config, f, indent=4, ensure_ascii=False)
        current_paths = config['paths']['local']
        print("💻 로컬 환경으로 전환되었습니다.")

# 필요한 디렉토리 생성
print("\n📁 필요한 디렉토리 생성 중...")
for key, path_str in current_paths.items():
    if key != 'base_path':
        path = Path(path_str)
        path.mkdir(parents=True, exist_ok=True)
        print(f"  ✅ {path}")

print("\n🎉 환경 설정이 완료되었습니다!")


In [ ]:
#@title 📥 2단계: 필요한 라이브러리 설치 { display-mode: "form" }
#@markdown 필요한 라이브러리들을 설치합니다.

import os
import subprocess
from pathlib import Path

print("📥 필요한 라이브러리 설치 중...")

# GitHub 저장소 클론 (로컬 환경이 아닌 경우)
try:
    if not Path('src').exists():  # src 폴더가 없으면 클론 필요
        print("🔄 GitHub 저장소 클론 중...")
        subprocess.run(['git', 'clone', 'https://github.com/doyoung42/chipchat_demo.git', 'temp_clone'], check=True)
        
        # 필요한 파일들 복사
        import shutil
        if Path('temp_clone/src').exists():
            shutil.copytree('temp_clone/src', 'src', dirs_exist_ok=True)
        if Path('temp_clone/requirements.txt').exists():
            shutil.copy2('temp_clone/requirements.txt', 'requirements.txt')
        
        # 임시 폴더 삭제
        shutil.rmtree('temp_clone')
        print("✅ 필요한 파일들 복사 완료")
    else:
        print("✅ 소스 코드 이미 존재")
except Exception as e:
    print(f"❌ GitHub 클론 실패: {str(e)}")
    print("💡 로컬 환경에서 이미 파일들이 존재한다고 가정합니다.")

# requirements.txt 설치
if Path('requirements.txt').exists():
    try:
        subprocess.run(['pip', 'install', '-r', 'requirements.txt', '-q'], check=True)
        print("✅ Requirements 설치 완료")
    except Exception as e:
        print(f"❌ Requirements 설치 실패: {str(e)}")
else:
    print("⚠️ requirements.txt 파일을 찾을 수 없습니다.")
    print("💡 기본 라이브러리들만 사용합니다.")

# 추가 패키지 설치 (Colab 전용)
try:
    from google.colab import drive
    subprocess.run(['pip', 'install', 'pyngrok==7.0.1', '-q'], check=True)
    print("✅ pyngrok 설치 완료")
except ImportError:
    print("💻 로컬 환경: pyngrok 설치 생략")
except Exception as e:
    print(f"❌ pyngrok 설치 실패: {str(e)}")

# Python path 설정
import sys
current_dir = Path(os.getcwd())
sys.path.append(str(current_dir))
sys.path.append(str(current_dir / 'src'))

# 새로운 디렉토리 구조 추가
src_dir = current_dir / 'src'
if src_dir.exists():
    sys.path.append(str(src_dir))
    for subdir in ['config', 'models', 'utils', 'app']:
        subdir_path = src_dir / subdir
        if subdir_path.exists():
            sys.path.append(str(subdir_path))

print("\n✅ 라이브러리 설치 및 경로 설정 완료!")
print("\n📋 지원되는 LLM 모델:")
print("🔸 OpenAI: gpt-4o-mini, gpt-4o, gpt-3.5-turbo")
print("🔸 Claude: claude-3-sonnet, claude-3-haiku, claude-3-opus")

In [ ]:
#@title 🚀 2-1단계: 로깅 시스템 초기화 및 모델 사전 다운로드 { display-mode: "form" }
#@markdown HuggingFace 모델을 캐싱하여 빠른 로딩을 지원합니다.

import time
from src.utils.logger import get_logger
from src.utils.model_cache import get_model_cache

# 로깅 시스템 초기화
print("📊 로깅 시스템 초기화 중...")
logger = get_logger()
logger.log_system_info()
logger.info("Main notebook 실행 시작")

# 모델 캐시 시스템 초기화
print("\n🗄️ 모델 캐시 시스템 초기화 중...")
model_cache = get_model_cache()
cache_info = model_cache.get_cache_info()

print(f"📁 캐시 디렉토리: {cache_info['cache_dir']}")
print(f"📦 캐시된 모델 수: {cache_info['model_count']}")
print(f"💾 전체 캐시 크기: {cache_info['total_size_mb']:.2f} MB")

if cache_info['models']:
    print("\n✅ 캐시된 모델:")
    for model in cache_info['models']:
        print(f"  - {model}")

# 모델 사전 다운로드 옵션
download_model = True #@param {type:"boolean"}
model_name = "sentence-transformers/all-MiniLM-L6-v2" #@param {type:"string"}

if download_model:
    print(f"\n🔄 모델 다운로드 또는 캐시 확인: {model_name}")
    
    @logger.measure_time("모델 준비")
    def prepare_model():
        # 캐시 확인
        if model_cache.is_model_cached(model_name):
            print("✅ 모델이 이미 캐시되어 있습니다.")
            # 캐시에서 로드
            if model_cache.load_model_from_cache(model_name):
                print("✅ 캐시에서 모델 로드 완료!")
                return True
        
        # 캐시에 없으면 다운로드
        print("📥 모델을 다운로드합니다...")
        try:
            from langchain_huggingface import HuggingFaceEmbeddings
            
            # 모델 다운로드 (초기화를 통해)
            start_time = time.time()
            embeddings = HuggingFaceEmbeddings(
                model_name=model_name,
                model_kwargs={'device': 'cpu'}
            )
            download_time = time.time() - start_time
            
            print(f"✅ 모델 다운로드 완료! (소요 시간: {download_time:.2f}초)")
            
            # 캐시에 저장
            print("💾 모델을 캐시에 저장 중...")
            if model_cache.save_model_to_cache(model_name):
                print("✅ 모델 캐시 저장 완료!")
            else:
                print("⚠️ 모델 캐시 저장 실패 (다음 실행 시 다시 다운로드됩니다)")
            
            return True
            
        except Exception as e:
            logger.error(f"모델 준비 실패: {e}")
            print(f"❌ 모델 준비 실패: {e}")
            return False
    
    # 모델 준비 실행
    success = prepare_model()
    
    if success:
        print("\n🎉 모델 준비 완료! 이제 Streamlit 앱이 더 빠르게 로드됩니다.")
    else:
        print("\n⚠️ 모델 준비에 실패했습니다. 하지만 계속 진행할 수 있습니다.")

# 성능 요약 표시
print("\n📊 현재까지의 성능 요약:")
perf_summary = logger.get_performance_summary()
for op, stats in perf_summary.items():
    print(f"\n🔹 {op}:")
    print(f"   - 평균 시간: {stats['avg_time']:.2f}초")
    print(f"   - 성공: {stats['success_count']}회, 실패: {stats['fail_count']}회")

print("\n✅ 모든 준비가 완료되었습니다!")

In [ ]:
#@title 🔑 3단계: API 키 설정 { display-mode: "form" }
#@markdown AI 서비스의 API 키를 입력하여 챗봇을 설정합니다.

import os
import json
from pathlib import Path

openai_api_key = "" #@param {type:"string"}
claude_api_key = "" #@param {type:"string"}
hf_token = "" #@param {type:"string"}

# API 키 유효성 검사
if not openai_api_key and not claude_api_key and not hf_token:
    print("⚠️ API 키가 입력되지 않았습니다.")
    print("💡 최소 하나 이상의 API 키를 입력해주세요.")
    print("• OpenAI API 키: https://platform.openai.com/api-keys")
    print("• Claude API 키: https://console.anthropic.com/keys")
    print("• HuggingFace 토큰: https://huggingface.co/settings/tokens")
else:
    # 환경 변수 설정
    if openai_api_key:
        os.environ["OPENAI_API_KEY"] = openai_api_key
        print("✅ OpenAI API 키 설정 완료!")
        
    if claude_api_key:
        os.environ["ANTHROPIC_API_KEY"] = claude_api_key
        print("✅ Claude API 키 설정 완료!")
    
    if hf_token:
        os.environ["HF_TOKEN"] = hf_token
        print("✅ HuggingFace 토큰 설정 완료!")
    
    # Streamlit 시크릿 파일 생성
    secrets_dir = Path(".streamlit")
    secrets_dir.mkdir(exist_ok=True)
    
    secrets = {}
    if openai_api_key:
        secrets["openai_api_key"] = openai_api_key
    if claude_api_key:
        secrets["anthropic_api_key"] = claude_api_key
    if hf_token:
        secrets["hf_token"] = hf_token
    
    with open(secrets_dir / "secrets.toml", "w") as f:
        for key, value in secrets.items():
            f.write(f'{key} = "{value}"\n')
    
    # 토큰 저장
    try:
        from src.config.token_manager import TokenManager
        token_manager = TokenManager()
        
        if openai_api_key:
            token_manager.set_token('openai', openai_api_key)
        
        if claude_api_key:
            token_manager.set_token('anthropic', claude_api_key)
        
        if hf_token:
            token_manager.set_token('huggingface', hf_token)
            
        print("✅ 토큰 관리자에 API 키 저장 완료!")
    except Exception as e:
        print(f"❌ 토큰 저장 중 오류가 발생했습니다: {str(e)}")
    
    # 기본 프롬프트 템플릿 생성
    default_template = {
        "pre": "당신은 전자 부품 데이터시트에 대해 응답하는 도우미입니다. 제공된 컨텍스트 정보를 기반으로 질문에 답변하세요.",
        "post": "검색된 정보를 바탕으로 명확하고 간결하게 답변해주세요."
    }
    
    # config.json에서 경로 읽기
    try:
        from src.utils.config_manager import get_config_manager
        config = get_config_manager()
        prompt_templates_folder = Path(config.get_path('prompt_templates_folder'))
    except:
        prompt_templates_folder = Path('./prompt_templates')
    
    template_file = prompt_templates_folder / "default_template.json"
    if not template_file.exists():
        with open(template_file, 'w', encoding='utf-8') as f:
            json.dump(default_template, f, ensure_ascii=False, indent=2)
            
        print("✅ 기본 프롬프트 템플릿이 생성되었습니다.")

In [ ]:
#@title 📊 4단계: Streamlit 서버 실행 { display-mode: "form" }

import subprocess
import threading
import time
import os
import json
import requests
from pathlib import Path

# 간소화된 ChipChat 앱 실행
app_file = "src/app/streamlit_app.py"
port = 8501

print(f"🎯 ChipChat 멀티턴 챗봇 실행")
print(f"📝 설명: LangGraph 기반 멀티에이전트 챗봇")
print(f"📁 파일: {app_file}")
print(f"🌐 포트: {port}")
print(f"")
print(f"✨ 주요 기능:")
print(f"  • 🤖 자동 도구 선택 (ChipDB, Vectorstore)")
print(f"  • 💬 멀티턴 대화 지원")
print(f"  • 🔧 LLM 모델 선택 가능")
print(f"  • 📊 실시간 성능 모니터링")

# config.json에서 경로 읽기
try:
    from src.utils.config_manager import get_config_manager
    config = get_config_manager()
    vectorstore_folder = config.get_path('vectorstore_folder')
    prep_json_folder = config.get_path('prep_json_folder')
    prompt_templates_folder = config.get_path('prompt_templates_folder')
except:
    vectorstore_folder = './vectorstore'
    prep_json_folder = './prep_json'
    prompt_templates_folder = './prompt_templates'

# 환경 변수 설정
os.environ['VECTORSTORE_PATH'] = str(vectorstore_folder)
os.environ['JSON_FOLDER_PATH'] = str(prep_json_folder)
os.environ['PROMPT_TEMPLATES_PATH'] = str(prompt_templates_folder)

# 로깅 정보 표시
if 'logger' in globals():
    logger.info("Streamlit 앱 시작 준비", extra={
        "vectorstore_path": str(vectorstore_folder),
        "json_folder_path": str(prep_json_folder),
        "prompt_templates_path": str(prompt_templates_folder),
        "port": port
    })

# 로그 출력 함수
def print_output(process):
    """실시간으로 프로세스 출력을 표시 (중요한 로그만)"""
    important_keywords = [
        'You can now view your Streamlit app',
        'Local URL:',
        'Network URL:', 
        'External URL:',
        'ERROR',
        'CRITICAL',
        'WARNING',
        'started server',
        'Server started',
        'Running on',
        'ModuleNotFoundError',
        'ImportError',
        'Exception'
    ]
    
    try:
        for line in iter(process.stdout.readline, ''):
            if line:
                line_text = line.strip() if isinstance(line, str) else line.decode('utf-8').strip()
                
                # 중요한 키워드가 포함된 로그만 표시
                if line_text and any(keyword.lower() in line_text.lower() for keyword in important_keywords):
                    print(f"📋 [Streamlit] {line_text}")
                elif 'streamlit' in line_text.lower() and ('ready' in line_text.lower() or 'starting' in line_text.lower()):
                    print(f"📋 [Streamlit] {line_text}")
    except Exception as e:
        print(f"⚠️ 로그 출력 중 오류: {e}")

# 서버 상태 확인 함수
def check_server_status(port, max_attempts=30):
    """서버가 실제로 시작되었는지 확인"""
    print(f"🔍 포트 {port}에서 서버 시작 상태를 확인 중...")
    for attempt in range(max_attempts):
        try:
            response = requests.get(f"http://localhost:{port}", timeout=2)
            if response.status_code == 200:
                print(f"✅ 서버가 정상적으로 시작되었습니다! (시도 {attempt + 1}/{max_attempts})")
                return True
        except requests.exceptions.RequestException:
            if attempt < max_attempts - 1:
                print(f"⏳ 서버 시작 대기 중... ({attempt + 1}/{max_attempts})")
                time.sleep(2)
            else:
                print(f"❌ 서버 시작 확인 실패 (시도 {attempt + 1}/{max_attempts})")
    return False

# Streamlit 실행 함수
def run_streamlit_unified(app_file, port):
    print("🔧 Streamlit 명령어 설정 중...")
    cmd = [
        "streamlit", "run", 
        app_file,
        f"--server.port={port}", 
        "--server.address=0.0.0.0",
        "--server.headless=true",
        "--browser.serverAddress=localhost",
        "--browser.gatherUsageStats=false",
        "--logger.level=info"
    ]
    
    print(f"📋 실행 명령어: {' '.join(cmd)}")
    print("🚀 Streamlit 프로세스 시작...")
    
    process = subprocess.Popen(
        cmd, 
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1
    )
    
    return process

# ngrok 사용 여부 선택
use_ngrok = False #@param {type:"boolean"}
ngrok_token = "" #@param {type:"string"}

print("\n📝 현재 설정:")
print(f"  • 포트: {port}")
print(f"  • Vectorstore 경로: {vectorstore_folder}")
print(f"  • JSON 폴더 경로: {prep_json_folder}")
print(f"  • 프롬프트 템플릿 경로: {prompt_templates_folder}")
print(f"  • ngrok 사용: {use_ngrok}")

# 기존 Streamlit 프로세스 종료
print("\n🔄 기존 Streamlit 프로세스 정리 중...")
try:
    subprocess.run(['pkill', '-f', 'streamlit'], check=False, capture_output=True)
    time.sleep(2)
    print("✅ 기존 프로세스 정리 완료")
except Exception as e:
    print(f"⚠️ 프로세스 정리 중 오류 (무시 가능): {e}")

# 통합 Streamlit 서버 실행
process = run_streamlit_unified(app_file, port)

# 별도 스레드에서 로그 출력
log_thread = threading.Thread(target=print_output, args=(process,), daemon=True)
log_thread.start()

# 서버 시작 상태 확인
server_started = check_server_status(port)

if server_started:
    if use_ngrok and ngrok_token:
        # ngrok 사용 (외부 접속 가능한 공개 URL)
        try:
            print("\n🌐 ngrok 터널 생성 중...")
            from pyngrok import ngrok
            ngrok.set_auth_token(ngrok_token)
            public_url = ngrok.connect(port).public_url
            print(f"✅ ngrok 터널 생성 완료!")
            print(f"\n🔗 다음 URL을 통해 ChipChat에 접속할 수 있습니다:")
            print(f"📱 {public_url}")
            print("\n📌 이 URL은 세션이 유지되는 동안에만 유효합니다.")
        except Exception as e:
            print(f"❌ ngrok 연결 실패: {str(e)}")
            print("🔄 Google Colab 내장 기능으로 전환합니다.")
            use_ngrok = False

    if not use_ngrok:
        # Google Colab 내장 기능 사용
        print(f"\n✅ 서버가 성공적으로 시작되었습니다!")
        print(f"\n🔗 ChipChat에 접속하는 방법:")
        print(f"1️⃣ 포트 {port}에서 실행 중입니다")
        print(f"2️⃣ 아래 버튼을 클릭하거나 왼쪽 사이드바의 '포트' 탭을 확인하세요")
        
        # 새창에서 열기
        try:
            from google.colab import output
            print("🖥️ 새창에서 ChipChat을 열 수 있는 버튼을 생성 중...")
            output.serve_kernel_port_as_window(port)
            print("✅ 새창 열기 버튼이 생성되었습니다!")
            print("🔘 위에 나타난 버튼을 클릭하면 새창에서 ChipChat이 열립니다.")
        except Exception as e:
            print(f"⚠️ 새창 버튼 생성 실패: {e}")
            print("💡 대신 다음 방법들을 시도해보세요:")
            print(f"   • 왼쪽 사이드바 → '포트' 탭 → {port} 포트 클릭")
            print(f"   • 직접 URL 접속: https://localhost:{port} (Colab 환경에서)")
    
    # 통합 앱 사용 안내
    print(f"\n💡 ChipChat 통합 앱 사용 방법:")
    print(f"  • 🎯 앱이 시작되면 모드를 선택하세요")
    print(f"  • ⚡ 경량 모드: 즉시 사용 가능 (기본 텍스트 검색)")
    print(f"  • 🤖 전체 모드: AI 기능 포함 (초기화 버튼 클릭 필요)")
    print(f"  • 🔄 언제든지 모드 전환 가능")
    print(f"  • 📊 시스템 상태는 메인 화면에서 확인")
    
else:
    print("\n❌ 서버 시작에 실패했습니다.")
    print("🔍 가능한 원인:")
    print("  • 필요한 파일이 누락되었을 수 있습니다")
    print("  • API 키 설정에 문제가 있을 수 있습니다")
    print("  • 의존성 라이브러리 설치에 문제가 있을 수 있습니다")
    print("\n💡 해결 방법:")
    print("  • 앱이 시작되면 경량 모드를 먼저 시도해보세요")
    print("  • 2-3단계를 다시 실행해보세요")
    print("  • 런타임을 재시작하고 처음부터 다시 시도해보세요")

# 프로세스 저장 (5단계에서 사용)
globals()['streamlit_process'] = process

In [ ]:
#@title 🛑 5단계: 서버 중지 { display-mode: "form" }
#@markdown 작업을 마치면 서버를 중지합니다.

import os
import signal
import subprocess
import time

# 서버 중지 여부 확인
stop_server = True  #@param {type:"boolean"}

if stop_server:
    print("🛑 서버 종료 프로세스를 시작합니다...")
    
    # ngrok 터널 종료 (ngrok 사용한 경우에만)
    try:
        from pyngrok import ngrok
        ngrok.kill()
        print("✅ ngrok 터널이 종료되었습니다.")
    except ImportError:
        pass
    except Exception as e:
        print(f"ℹ️ ngrok 종료: {str(e)}")
    
    # 4단계에서 생성된 프로세스 먼저 종료
    terminated_process = False
    if 'streamlit_process' in globals():
        try:
            process = globals()['streamlit_process']
            if process and process.poll() is None:
                print("🔄 4단계에서 실행된 Streamlit 프로세스를 종료 중...")
                process.terminate()
                time.sleep(3)
                
                if process.poll() is None:
                    print("⚡ 강제 종료를 시도합니다...")
                    process.kill()
                    time.sleep(2)
                
                print("✅ Streamlit 프로세스가 정상적으로 종료되었습니다.")
                terminated_process = True
            else:
                print("ℹ️ Streamlit 프로세스가 이미 종료되었습니다.")
        except Exception as e:
            print(f"⚠️ 프로세스 종료 중 오류: {e}")
    
    # 추가적으로 모든 streamlit 프로세스 종료
    try:
        print("🔍 남은 Streamlit 프로세스를 검색 중...")
        result = subprocess.run(['pgrep', '-f', 'streamlit'], capture_output=True, text=True)
        if result.stdout.strip():
            pids = result.stdout.strip().split('\n')
            print(f"📋 발견된 Streamlit 프로세스: {len(pids)}개")
            
            subprocess.run(['pkill', '-f', 'streamlit'], check=False)
            time.sleep(2)
            
            result_after = subprocess.run(['pgrep', '-f', 'streamlit'], capture_output=True, text=True)
            if not result_after.stdout.strip():
                print("✅ 모든 Streamlit 프로세스가 정리되었습니다.")
            else:
                print("⚠️ 일부 프로세스가 여전히 실행 중일 수 있습니다.")
        else:
            print("✅ 실행 중인 Streamlit 프로세스가 없습니다.")
            
    except Exception as e:
        print(f"⚠️ 프로세스 정리 중 오류가 발생했습니다: {e}")
        print("💡 수동으로 런타임을 재시작하여 서버를 종료할 수 있습니다.")
    
    # 포트 사용 상태 확인
    try:
        import socket
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('localhost', 8501))
        sock.close()
        
        if result != 0:
            print("✅ 포트 8501이 해제되었습니다.")
        else:
            print("⚠️ 포트 8501이 아직 사용 중입니다.")
    except Exception as e:
        print(f"ℹ️ 포트 상태 확인 실패: {e}")
    
    print("\n📋 서버 종료 완료!")
    print("💡 완전한 정리를 위해 런타임을 재시작하는 것을 권장합니다.")
    print("🔄 런타임 → 세션 재시작을 통해 모든 프로세스를 완전히 종료할 수 있습니다.")
else:
    print("ℹ️ 서버 중지가 취소되었습니다.")
    print("💡 서버를 중지하려면 위의 체크박스를 선택하고 다시 실행해주세요.")

## 🛠️ 문제 해결

**애플리케이션이 실행되지 않는 경우:**

1. **API 키 확인**: OpenAI API 키가 올바르게 입력되었는지 확인
2. **세션 재시작**: 런타임 → 세션 재시작 후 처음부터 다시 실행
3. **JSON 파일 확인**: 전처리된 JSON 파일이 올바른 경로에 존재하는지 확인

**사용 중 문제가 발생하는 경우:**
- 브라우저를 새로고침하세요
- 네트워크 연결 상태를 확인하세요
